In [3]:
import requests
import json
import urllib.request, urllib.error, urllib.parse
import pprint
import time

api_key = 'awPgd5YOKYVv0DCC8BERPRPeKJhXmGw4'

docs = []

def get_articles(query, api_key, page=0, filter_params='&facet=true&begin_date=20170101&end_date=20240901'):
    # Adding filter query (fq) for more precise control
    fq = 'fq=body:("Zara") AND body:("clothing") AND NOT body:("Trump") AND NOT body:("Israel")'
    url = f"https://api.nytimes.com/svc/search/v2/articlesearch.json?q={query}&page={page}&api-key={api_key}&{fq}{filter_params}"
    response = requests.get(url)
    response.raise_for_status()  # Raise an error for bad responses
    return response.json()

for i in range(10): # For 100 articles
    print(i)
    articles = get_articles("Zara", api_key, page = i)
    x = articles['response']['docs']
    docs.extend(x)
    time.sleep(12)

docs
    
text = []
for doc in docs:
    text.append(doc['headline']['main'] + '. ' +  doc['abstract'])
text

def read_scores(sent_file):
    with open(sent_file) as f:
        return {line.split('\t')[0]: int(line.split('\t')[1]) for line in f}
Scores_Table = read_scores('Files_Directory/Twitter/AFINN-111.txt')
# Scores_Table = read_scores('AFINN-111.txt')

scores = []

for txt in text:
    count = 0;
    for score in Scores_Table:
         string = json.dumps(txt).lower()
         word_score = string.count(score) * Scores_Table[score]
         count = count + word_score
    scores.append(count)
!pip install azure-ai-textanalytics

import requests
import azure
import time

key = 'ce14ce8f012d462fbbedd4c70565b678' #"<paste-your-text-analytics-key-here>"
endpoint = 'https://fischba-webdataproject-fall2024.openai.azure.com/' #"<paste-your-text-analytics-endpoint-here>"


from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential


def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint,
            credential=ta_credential)
    return text_analytics_client


def sentiment_analysis_example(client, txt):
    documents = [txt]
    response = client.analyze_sentiment(documents=documents)[0]
    print(response)
    responses_all.append(response)

def key_phrase_extraction_example(client, txt):

    try:
        documents = [txt]

        response = client.extract_key_phrases(documents = documents)[0]

        if not response.is_error:
            return response.key_phrases
        else:
            print(response.id, response.error)

    except Exception as err:
        print("Encountered exception. {}".format(err))

azure_keyphrases = []
azure_sentiments = []
responses_all =[]
txt_number = 0
requests_per_second_limit = 100
time_counter = time.time()  # Start time of processing the request
for txt in text:
    txt_number+=1
    print(txt_number)
    client = authenticate_client()
 # Detect key phrases.
    key_phrases = key_phrase_extraction_example(client,txt)
    azure_keyphrases.append(key_phrases)
 # Detect Sentiments
    sentiments = sentiment_analysis_example(client, txt)

    end_time = time.time()  # End time of processing the request
    processing_time = end_time - time_counter

    requests_in_this_second = txt_number % requests_per_second_limit

    # If we reach the limit of requests per second, add a delay to maintain the limit
    if requests_in_this_second == 0:
        time_taken_for_batch = end_time - time_counter
        if time_taken_for_batch < 1:
            time.sleep(1 - time_taken_for_batch)
        time_counter = time.time()
        
final_sentiments=[]
for i in responses_all:
    x = i['sentiment']
    final_sentiments.append(x)
    
final_sentiments

print(final_sentiments.count('positive'))
print(final_sentiments.count('negative'))

0
1
2
3
4
5
6
7
8
9
1
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.01, neutral=0.88, negative=0.11), 'sentences': [SentenceSentiment(text=‘The Golden Age of Thrifting Is Over’. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.02, neutral=0.95, negative=0.03), length=39, offset=0, mined_opinions=[]), SentenceSentiment(text=A glut of fast fashion has made its way into local thrift stores, making it hard for women who have been purchasing secondhand for decades to find quality garments they can wear., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.81, negative=0.18), length=178, offset=39, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
2
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.04, neutral=0.89, negative=0.07), 'sentences': [Sent

{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.0, neutral=0.97, negative=0.02), 'sentences': [SentenceSentiment(text=Finding Clothes, and Identity, Outside Men’s and Women’s Wear. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=1.0, negative=0.0), length=63, offset=0, mined_opinions=[]), SentenceSentiment(text=More clothing companies are trying to meet the demands of a changing clientele, offering gender-bending, gender-fluid and gender-neutral items., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=0.95, negative=0.04), length=143, offset=63, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
11
{'id': '0', 'sentiment': 'negative', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.0, neutral=0.16, negative=0.83), 'sentences': [SentenceSentiment(text=In Britain, a ‘ret

{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.01, neutral=0.77, negative=0.22), 'sentences': [SentenceSentiment(text=Gucci and Adidas Apologize and Drop Products Called Racist. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=0.92, negative=0.08), length=60, offset=0, mined_opinions=[]), SentenceSentiment(text=Gucci removed a sweater that critics said evoked blackface imagery, while Adidas abandoned a white shoe it was marketing as part of a Black History Month tribute., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.63, negative=0.36), length=162, offset=60, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
22
{'id': '0', 'sentiment': 'negative', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.0, neutral=0.14, negative=0.86), 'sentences': [SentenceSentiment(tex

{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.22, neutral=0.77, negative=0.0), 'sentences': [SentenceSentiment(text=Open Thread: This Week in Style News. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=1.0, negative=0.0), length=38, offset=0, mined_opinions=[]), SentenceSentiment(text=The Golden Globes gets a new red carpet attitude, sneaker heads flock to Sneaker Con — and more., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.45, neutral=0.55, negative=0.0), length=96, offset=38, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
33
{'id': '0', 'sentiment': 'positive', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.51, neutral=0.49, negative=0.0), 'sentences': [SentenceSentiment(text=The Royal Baby: Frequently Asked Questions. , sentiment=neutral, confidence_scores=Sentimen

{'id': '0', 'sentiment': 'negative', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.0, neutral=0.49, negative=0.51), 'sentences': [SentenceSentiment(text=Retail Cornerstones Fall in Britain, Pushed by Fast Fashion and Pandemic. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=1.0, negative=0.0), length=74, offset=0, mined_opinions=[]), SentenceSentiment(text=Debenhams and Arcadia Group, the owner of Topshop, collapsed despite extensive government programs meant to shore up British businesses., sentiment=negative, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=0.49, negative=0.51), length=136, offset=74, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
44
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.01, neutral=0.83, negative=0.15), 'sentences': [SentenceSentiment(text=Shopping for

{'id': '0', 'sentiment': 'negative', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.0, neutral=0.0, negative=1.0), 'sentences': [SentenceSentiment(text=The True Cost of a $12 T-Shirt. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=1.0, negative=0.0), length=32, offset=0, mined_opinions=[]), SentenceSentiment(text=Ten years after the Rana Plaza factory disaster, which killed over 1,100 workers, millions of apparel workers face even worse conditions than before., sentiment=negative, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=0.0, negative=1.0), length=149, offset=32, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
54
{'id': '0', 'sentiment': 'negative', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.0, neutral=0.2, negative=0.8), 'sentences': [SentenceSentiment(text=How Much Did That Zipper Cost? , sentiment=neut

{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.03, neutral=0.94, negative=0.03), 'sentences': [SentenceSentiment(text=Amid the Renaissance Architecture of Florence, an Unexpected Villa. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.05, neutral=0.93, negative=0.02), length=68, offset=0, mined_opinions=[]), SentenceSentiment(text=For their debut project in the city, Dimore Studio managed to transform a neglected 1960s-era house into a deeply contemporary take on Italian Modernism., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=0.95, negative=0.05), length=153, offset=68, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
65
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.08, neutral=0.92, negative=0.0), 'sentences': [SentenceSentiment(text=

{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.01, neutral=0.83, negative=0.15), 'sentences': [SentenceSentiment(text=Decoding the Surprise Inflation Report. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.02, neutral=0.91, negative=0.07), length=40, offset=0, mined_opinions=[]), SentenceSentiment(text=A stubbornly high inflation reading has divided experts on what the Fed should do when it weighs another interest rate hike next week., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.75, negative=0.24), length=134, offset=40, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
76
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.02, neutral=0.97, negative=0.0), 'sentences': [SentenceSentiment(text=Will We Ever Be Able to Recycle Our Clothes Li

{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.01, neutral=0.94, negative=0.05), 'sentences': [SentenceSentiment(text=To Survive the Pandemic, Savile Row Cuts a Bespoke Strategy. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.98, negative=0.02), length=61, offset=0, mined_opinions=[]), SentenceSentiment(text=With formal events and travel canceled, the tradition-bound tailors are gently embracing technology — and finding leverage on their landlords., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.91, negative=0.08), length=142, offset=61, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
87
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.02, neutral=0.88, negative=0.1), 'sentences': [SentenceSentiment(text=Strippers Are Doi

In [4]:
import requests
import json
import urllib.request, urllib.error, urllib.parse
import pprint
import time

api_key = 'awPgd5YOKYVv0DCC8BERPRPeKJhXmGw4'

docs = []

def get_articles(query, api_key, page=0, filter_params='&facet=true&begin_date=20170101&end_date=20240901'):
    # Adding filter query (fq) for more precise control
    fq = 'fq=body:("H%26M") AND body:("clothing") AND NOT body:("Trump") AND NOT body:("Israel")'
    url = f"https://api.nytimes.com/svc/search/v2/articlesearch.json?q={query}&page={page}&api-key={api_key}&{fq}{filter_params}"
    response = requests.get(url)
    response.raise_for_status()  # Raise an error for bad responses
    return response.json()

for i in range(10): # For 100 articles
    print(i)
    articles = get_articles("H%26M", api_key, page = i)
    x = articles['response']['docs']
    docs.extend(x)
    time.sleep(12)

docs
    
text = []
for doc in docs:
    text.append(doc['headline']['main'] + '. ' +  doc['abstract'])
text

def read_scores(sent_file):
    with open(sent_file) as f:
        return {line.split('\t')[0]: int(line.split('\t')[1]) for line in f}
Scores_Table = read_scores('Files_Directory/Twitter/AFINN-111.txt')
# Scores_Table = read_scores('AFINN-111.txt')

scores = []

for txt in text:
    count = 0;
    for score in Scores_Table:
         string = json.dumps(txt).lower()
         word_score = string.count(score) * Scores_Table[score]
         count = count + word_score
    scores.append(count)
!pip install azure-ai-textanalytics

import requests
import azure
import time

key = 'ce14ce8f012d462fbbedd4c70565b678' #"<paste-your-text-analytics-key-here>"
endpoint = 'https://fischba-webdataproject-fall2024.openai.azure.com/' #"<paste-your-text-analytics-endpoint-here>"


from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential


def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint,
            credential=ta_credential)
    return text_analytics_client


def sentiment_analysis_example(client, txt):
    documents = [txt]
    response = client.analyze_sentiment(documents=documents)[0]
    print(response)
    responses_all.append(response)

def key_phrase_extraction_example(client, txt):

    try:
        documents = [txt]

        response = client.extract_key_phrases(documents = documents)[0]

        if not response.is_error:
            return response.key_phrases
        else:
            print(response.id, response.error)

    except Exception as err:
        print("Encountered exception. {}".format(err))

azure_keyphrases = []
azure_sentiments = []
responses_all =[]
txt_number = 0
requests_per_second_limit = 100
time_counter = time.time()  # Start time of processing the request
for txt in text:
    txt_number+=1
    print(txt_number)
    client = authenticate_client()
 # Detect key phrases.
    key_phrases = key_phrase_extraction_example(client,txt)
    azure_keyphrases.append(key_phrases)
 # Detect Sentiments
    sentiments = sentiment_analysis_example(client, txt)

    end_time = time.time()  # End time of processing the request
    processing_time = end_time - time_counter

    requests_in_this_second = txt_number % requests_per_second_limit

    # If we reach the limit of requests per second, add a delay to maintain the limit
    if requests_in_this_second == 0:
        time_taken_for_batch = end_time - time_counter
        if time_taken_for_batch < 1:
            time.sleep(1 - time_taken_for_batch)
        time_counter = time.time()
        
final_sentiments=[]
for i in responses_all:
    x = i['sentiment']
    final_sentiments.append(x)
    
final_sentiments

print(final_sentiments.count('positive'))
print(final_sentiments.count('negative'))

0
1
2
3
4
5
6
7
8
9
1
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.0, neutral=0.99, negative=0.01), 'sentences': [SentenceSentiment(text=H&M’s Different Kind of Clickbait. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=1.0, negative=0.0), length=35, offset=0, mined_opinions=[]), SentenceSentiment(text=The Swedish retailer now lets customers know where nearly every garment it sells is made. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=1.0, negative=0.0), length=90, offset=35, mined_opinions=[]), SentenceSentiment(text=Is that enough?, sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.97, negative=0.02), length=15, offset=125, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
2
{'id': '0', 'sentiment': 'negative', 'warnings': [], 'statistics': None, 'confidence_scores

{'id': '0', 'sentiment': 'positive', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.57, neutral=0.43, negative=0.0), 'sentences': [SentenceSentiment(text=Calling All Bodies. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.98, negative=0.0), length=20, offset=0, mined_opinions=[]), SentenceSentiment(text=With a graphic new H&M capsule collection and joyful accompanying video, Mugler celebrates the inclusivity the label’s founder pioneered., sentiment=positive, confidence_scores=SentimentConfidenceScores(positive=0.57, neutral=0.43, negative=0.0), length=137, offset=20, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
11
{'id': '0', 'sentiment': 'negative', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.0, neutral=0.26, negative=0.74), 'sentences': [SentenceSentiment(text=Is Brazilian Leather Out of Fashion? , sentiment=neutral, confi

{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.01, neutral=0.88, negative=0.11), 'sentences': [SentenceSentiment(text=G.M. Used Graffiti in a Car Ad. Should the Artist Be Paid?. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.02, neutral=0.82, negative=0.16), length=60, offset=0, mined_opinions=[]), SentenceSentiment(text=As graffiti becomes a marketing tool, copyright law has struggled to catch up. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=0.85, negative=0.15), length=79, offset=60, mined_opinions=[]), SentenceSentiment(text=A lawsuit could decide when an artist is entitled to compensation., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.98, negative=0.02), length=66, offset=139, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
22
{'id': '0', 'sentiment': 'mixed', 'warnin

{'id': '0', 'sentiment': 'positive', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.98, neutral=0.01, negative=0.02), 'sentences': [SentenceSentiment(text=Shopping for Laundry Hampers. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.04, neutral=0.96, negative=0.0), length=30, offset=0, mined_opinions=[]), SentenceSentiment(text=The best ones make dealing with dirty laundry a whole lot easier., sentiment=positive, confidence_scores=SentimentConfidenceScores(positive=0.98, neutral=0.01, negative=0.02), length=65, offset=30, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
33
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.0, neutral=0.7, negative=0.29), 'sentences': [SentenceSentiment(text=Bangladesh’s Crackdown on Labor. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=0.67, 

{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.07, neutral=0.93, negative=0.0), 'sentences': [SentenceSentiment(text=How Fashion Giants Recast Plastic as Good for the Planet. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.12, neutral=0.88, negative=0.0), length=58, offset=0, mined_opinions=[]), SentenceSentiment(text=An influential system overseen by retailers and clothing makers ranks petroleum-based synthetics like “vegan leather” as more environmentally sound than natural fibers., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.98, negative=0.0), length=168, offset=58, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
44
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.0, neutral=0.99, negative=0.01), 'sentences': [SentenceSentiment(te

{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.01, neutral=0.96, negative=0.03), 'sentences': [SentenceSentiment(text=The People’s Republic of Shein. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=1.0, negative=0.0), length=32, offset=0, mined_opinions=[]), SentenceSentiment(text=Shein continues its rise with American shoppers, who don’t mind the controversies., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.03, neutral=0.92, negative=0.05), length=82, offset=32, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
55
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.02, neutral=0.97, negative=0.0), 'sentences': [SentenceSentiment(text=Will We Ever Be Able to Recycle Our Clothes Like an Aluminum Can?. , sentiment=neutral, confidence_scores=Sent

{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.12, neutral=0.88, negative=0.01), 'sentences': [SentenceSentiment(text=The California Fur Ban and What It Means for You. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.99, negative=0.0), length=50, offset=0, mined_opinions=[]), SentenceSentiment(text=Is this the beginning of the end of the mink coat? , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=0.98, negative=0.01), length=51, offset=50, mined_opinions=[]), SentenceSentiment(text=Here’s what you need to know., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.34, neutral=0.66, negative=0.0), length=29, offset=101, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
66
{'id': '0', 'sentiment': 'negative', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScore

{'id': '0', 'sentiment': 'mixed', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.29, neutral=0.33, negative=0.38), 'sentences': [SentenceSentiment(text=This Is the Reality of America’s Fast-Fashion Addiction. , sentiment=positive, confidence_scores=SentimentConfidenceScores(positive=0.58, neutral=0.42, negative=0.0), length=57, offset=0, mined_opinions=[]), SentenceSentiment(text=I took a job to study how a tech company’s focus on speed and scale affects  warehouse workers. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=1.0, negative=0.0), length=96, offset=57, mined_opinions=[]), SentenceSentiment(text=I found myself facing an onslaught of cheap clothing., sentiment=negative, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=0.24, negative=0.76), length=53, offset=153, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
76
{'id': '0', 'sentiment': 'negative', 'warnin

{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.01, neutral=0.99, negative=0.0), 'sentences': [SentenceSentiment(text=The Dress Doctor Is In. Dawnn Karen of the Fashion Institute of Technology is a leader in the growing field of fashion psychology — or why we want what we think we want., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.99, negative=0.0), length=169, offset=0, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
86
{'id': '0', 'sentiment': 'negative', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.09, neutral=0.39, negative=0.52), 'sentences': [SentenceSentiment(text=High-Skilled White-Collar Work? , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=0.99, negative=0.01), length=32, offset=0, mined_opinions=[]), SentenceSentiment(text=Machines Can Do That,

{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceScores(positive=0.01, neutral=0.98, negative=0.01), 'sentences': [SentenceSentiment(text=The State of the American Mall. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=1.0, negative=0.0), length=32, offset=0, mined_opinions=[]), SentenceSentiment(text=The pandemic has changed shopping habits. , sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.0, neutral=0.98, negative=0.02), length=42, offset=32, mined_opinions=[]), SentenceSentiment(text=Today, we take a look at what’s happening with malls in the U.S., sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.03, neutral=0.97, negative=0.0), length=64, offset=74, mined_opinions=[])], 'is_error': False, 'kind': 'SentimentAnalysis'}
96
{'id': '0', 'sentiment': 'neutral', 'warnings': [], 'statistics': None, 'confidence_scores': SentimentConfidenceS